Celem ćwiczenia jest oczyszczenie zbioru TitanicMess.tsv (przygotowanie pod analizy). W ćwiczeniu nie będą usuwane całe kolumny a jedynie modyfikowane.

In [1]:
import numpy as np
import pandas as pd

Na początek zaimportowano zbiór, wyświetlono pierwsze 15 wierszy oraz typy danych.

In [2]:
df = pd.read_csv("TitanicMess.tsv",sep="\t")
df.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q,Titanic
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S,Titanic
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S,Titanic
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S,Titanic
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,"30,0708",NaN,C,Titanic


In [3]:
df.dtypes

PassengerId     int64
Survived        int64
Pclass          int64
Name           object
Sex            object
Age            object
SibSp           int64
Parch           int64
Ticket         object
Fare           object
Cabin          object
Embarked       object
ship           object
dtype: object

Sprawdzenie poprawności danych dla kolumn, które zawierają dane powtarzające się. Pominięto kolumny, w których dane przyjmują różne wartości lub jest to ID ('PassengerId', 'Name', 'Ticket', 'Age', 'Fare', 'Cabin').

In [4]:
for col in df:
    if col in ['PassengerId', 'Name', 'Ticket', 'Age', 'Fare', 'Cabin']:
        continue
    print(col)
    print(df[col].unique())

Survived
[0 1]
Pclass
[3 1 2]
Sex
['male' 'female' 'malef' 'mal' 'fem' 'femmale']
SibSp
[1 0 3 4 2 5 8]
Parch
[0 1 2 5 3 4]
Embarked
['S' 'C' 'Q' 'So' nan 'Co' 'Qe']
ship
['Titanic']


W kolumnach 'Sex' oraz 'Embarked' zmieniono tekst tak, aby zawierał tylko pierwsze litery ciągu znaków. Ma to na celu usunięcie błędów, takich jak dodatkowe litery w oznaczeniu płci. Dla kolumny 'Embarked' przyjęto oznaczenie 'n' jako wartość brakującą.

In [5]:
df.Embarked=df.Embarked.astype(str)
for x in range (0, len(df)):
    df.Sex[x]=df.Sex[x][0]
    df.Embarked[x]=df.Embarked[x][0]

<ipython-input-5-117701e55122>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Sex[x]=df.Sex[x][0]
<ipython-input-5-117701e55122>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Embarked[x]=df.Embarked[x][0]


Zamiana typu danych dla kolumny 'Age'. Kolumna zawiera przecinek w postaci zarówno ',' jak i '.', zamieniono znak z ',' na '.'. Ma to na celu dostosowanie kolumny pod uzupełnienie wartości brakujących jako średnią. Bez tego kroku obliczanie wartości średniej spowoduje błąd, ze względu na typ danych. Uzupełnienie wartości 'Age' za pomocą średniej powinno w tym przypadku być bardziej wartościowe niż uzupełnienie wartością np. '0'. Jako typ zmiennej wybrano float, ze względu na możliwość dokładniejszego oznaczania wieku. Typ ten nie powinien wpłynąć na dokonywane na zbiorze obliczenia oraz łatwo ulega zamianie na typ całkowity.

In [6]:
df["Age"] = df["Age"].str.replace(",",".").astype(float)

In [7]:
df['Age']=df['Age'].fillna(df['Age'].mean())

Z kolumny 'Fare' usunięto przecinek oraz dodatkowe znaki, ponieważ jest to kwota wyrażona w funtach ("Fare is in Pre-1970 British Pounds" - źródło "http://campus.lakeforest.edu/frank/FILES/MLFfiles/Bio150/Titanic/TitanicMETA.pdf").

In [8]:
df["Fare"] = df["Fare"].str.replace(",","")
df['Fare'] = df['Fare'].str.extract('(\d+)', expand=False).astype(int)

Ponownie wyświetlono powtarzające się wartości oraz typy zmiennych. Ma to na celu przedstawienie oczyszczonego zbioru w przyjazny sposób.

In [9]:
for col in df:
    if col in ['PassengerId', 'Name', 'Ticket', 'Age', 'Fare', 'Cabin']:
        continue
    print(col)
    print(df[col].unique())

Survived
[0 1]
Pclass
[3 1 2]
Sex
['m' 'f']
SibSp
[1 0 3 4 2 5 8]
Parch
[0 1 2 5 3 4]
Embarked
['S' 'C' 'Q' 'n']
ship
['Titanic']


In [10]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare             int32
Cabin           object
Embarked        object
ship            object
dtype: object

Ostatnim krokiem jest zmiana ID pasażera w taki sposób aby liczby się nie powtarzały. Jak można zauważyć po wyświetleniu pierwszych 15 wierszy mamy wartość powtarzającą się (liczba 11). Id zostaje uzupełnione jako liczba całkowita rozpoczynając od 1 inkrementując o 1.

In [11]:
for x in range (0, len(df)):
    df['PassengerId'][x]=x+1
df.head(15)

<ipython-input-11-99edb33ade98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PassengerId'][x]=x+1


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",m,22.000000,1,0,A/5 21171,725,NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",f,38.000000,1,0,PC 17599,712833,C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",f,26.000000,0,0,STON/O2. 3101282,7925,NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",f,35.000000,1,0,113803,531,C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",m,35.000000,0,0,373450,805,NaN,S,Titanic
5,6,0,3,"Moran, Mr. James",m,35.929166,0,0,330877,84583,NaN,Q,Titanic
6,7,0,1,"McCarthy, Mr. Timothy J",m,54.000000,0,0,17463,518625,E46,S,Titanic
7,8,0,3,"Palsson, Master. Gosta Leonard",m,2.000000,3,1,349909,21075,NaN,S,Titanic
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",f,27.000000,0,2,347742,111333,NaN,S,Titanic
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",f,14.000000,1,0,237736,300708,NaN,C,Titanic


In [12]:
df.to_csv("TitanicCleaned.tsv",index=False, sep='\t')